In [1]:
!pip install yfinance pandas


In [2]:
import yfinance as yf
import pandas as pd


class TradingStrategy:
    def __init__(self, symbol, start_date, end_date, budget=5000):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.budget = budget

        self.cash = float(budget)
        self.shares = 0
        self.data = None

    def load_data(self):
        df = yf.download(
            self.symbol,
            start=self.start_date,
            end=self.end_date,
            auto_adjust=True
        )

        df = df.drop_duplicates()
        df = df.ffill()

        df["MA50"] = df["Close"].rolling(50).mean()
        df["MA200"] = df["Close"].rolling(200).mean()

        self.data = df

    def get_value(self, column, index):
        """Safely extract scalar value from DataFrame"""
        value = self.data[column].iloc[index]
        return float(value.values[0]) if hasattr(value, "values") else float(value)

    def run(self):
        self.load_data()

        for i in range(1, len(self.data)):
            prev_ma50 = self.get_value("MA50", i - 1)
            prev_ma200 = self.get_value("MA200", i - 1)
            curr_ma50 = self.get_value("MA50", i)
            curr_ma200 = self.get_value("MA200", i)
            price = self.get_value("Close", i)

            if pd.isna(prev_ma50) or pd.isna(prev_ma200):
                continue

            # BUY — Golden Cross
            if prev_ma50 <= prev_ma200 and curr_ma50 > curr_ma200 and self.shares == 0:
                self.shares = int(self.cash / price)
                self.cash -= self.shares * price
                print(f"🟢 BUY  {self.shares} shares at ${price:.2f}")

            # SELL — Death Cross
            elif prev_ma50 >= prev_ma200 and curr_ma50 < curr_ma200 and self.shares > 0:
                self.cash += self.shares * price
                print(f"🔴 SELL {self.shares} shares at ${price:.2f}")
                self.shares = 0

        # Force sell at end
        if self.shares > 0:
            last_price = self.get_value("Close", -1)
            self.cash += self.shares * last_price
            print(f"⚫ FORCE SELL {self.shares} shares at ${last_price:.2f}")
            self.shares = 0

        profit = self.cash - self.budget

        print("\n==============================")
        print(f"Final Balance : ${self.cash:.2f}")
        print(f"Profit / Loss : ${profit:.2f}")
        print("==============================")


# Run strategy
strategy = TradingStrategy("AAPL", "2018-01-01", "2023-12-31")
strategy.run()


[*********************100%***********************]  1 of 1 completed


🟢 BUY  100 shares at $49.77
🔴 SELL 100 shares at $142.78
🟢 BUY  96 shares at $148.28
🔴 SELL 96 shares at $137.78
🟢 BUY  85 shares at $155.72
⚫ FORCE SELL 85 shares at $190.73

Final Balance : $16268.63
Profit / Loss : $11268.63
